# Охваты каналов в домохозяйствах
Пример расчета накопленных охватов по домохозяйствам

## Описание задачи и условий расчета

- Период: 01.12.2022 - 31.12.2022
- Временной интервал: 05:00-29:00
- ЦА: Все домохозяйства
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index										
- Статистики: Reach(000), Reach%, Sample, Universe(000)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Расчет охватов
Сначала посчитаем охваты телеканалов
### Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-12-01', '2022-12-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['Reach000', 'ReachPer']

# Указываем срезы: телесети
slices = ['tvNetName']

# Задаем условия сортировки: телесеть (от а до я)
sortings = {"tvNetName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "useNbd": False, #Расчет охватов без nbd коррекции
    "viewingSubject": "HOUSEHOLD" #Расчет по домохозяйствам
}

### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Расчет Sample и Universe(000)
В этом разделе мы рассчитаем статистики Sample и Universe(000) и добавим их к таблице охватов.
Если вы хотите выгрузить таблицу в ее текущем виде, перейдите сразу к сохранению в Excel в конце ноутбука

### Формирование задания
Зададим условия расчета

In [ ]:
# Указываем список статистик для расчета
statistics = ['Sample', 'Universe000']

# Указываем срезы: без срезов
slices = []

# Задаем условия сортировки: без соритровки
sortings = None

### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_sample_uni = mtask.result2table(mtask.get_result(task_timeband))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df_sample_uni = df_sample_uni[slices+statistics]
df_sample_uni

### Объединяем таблицы
Добавим значения Sample и Universe(000) в ранее посчитанную таблицу охватов

In [ ]:
df['Sample'] = df_sample_uni.loc[0, 'Sample']
df['Universe000'] = df_sample_uni.loc[0, 'Universe000']

In [ ]:
df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('06_reach_by_households')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)